In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim

In [2]:
df = pd.read_excel (r'data.xlsx')

In [3]:
df = df[~df['Fallnummer'].isnull()]

In [4]:
class PatientClassificationNet(nn.Module):
    def __init__(self, input_dim, hidden_size, activation=torch.relu):
        super(PatientClassificationNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.activation = activation
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x)
        return x

In [5]:
X = df.loc[:,['AGE','Admission type 2','No. of times sent to ICU','FA ab para']]
X['FA ab para'] = X['FA ab para'].replace([1,5, 10, 14, 16, 21, 22], X['FA ab para'].max()+1)
X = torch.Tensor(X.to_numpy())
fa_val = X[:, 3].unique()
tmp = X[:,3].unsqueeze(1).expand((-1,len(fa_val))) == fa_val
X = torch.cat((X[:, :3], tmp.float()), dim=1)
X[:,2] = (X[:,2] > 1 )
X = (X - X.mean(0)) / X.std(0)

Y = df.loc[:,['Tod']]
Y = torch.Tensor(Y.to_numpy())

num_feature = X.shape[-1]

In [6]:
num_train = int(len(X) * 0.8)
shuffle_indices = np.arange(len(X))
np.random.seed(6)
np.random.shuffle(shuffle_indices)
X_train = X[shuffle_indices][:num_train]
y_train = Y[shuffle_indices][:num_train]
X_test = X[shuffle_indices][num_train:]
y_test = Y[shuffle_indices][num_train:]
dataset_train = TensorDataset(X_train, y_train)
dataset_test = TensorDataset(X_test, y_test)
trainloader = DataLoader(dataset_train, batch_size=128, shuffle=True)
testloader = DataLoader(dataset_test, batch_size=128, shuffle=True)

In [7]:
y_train.sum()

tensor(320.)

In [8]:
threshold = y_train.nonzero().size(0)/num_train

In [9]:
def train(net, trainloader, testloader, epoch=5):
    criterion = nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9, weight_decay=1e-6)
    for epoch in range(epoch):
        total_loss = 0
        net.train()
        for i, data in enumerate(trainloader):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
#             print(loss)
            loss.backward()
            optimizer.step()
            total_loss += loss
        
        net.eval()
        correct_true = 0
        predicted_true = 0
        target_true = 0
        total = 0
        for i, data in enumerate(testloader):
            inputs, labels = data
            outputs = net(inputs)
            prediction = outputs > threshold
            correct_true += ((prediction==1) * (labels==1)).sum().item()
            target_true += labels.sum().item()
            predicted_true += prediction.sum().item()
            total += labels.size(0)
        recall = correct_true / target_true
        precision = correct_true / predicted_true
        f1_score = 2 * precision * recall / (precision + recall)
        print(f'epoch {epoch}: recall: {recall} precision: {precision} f1_score: {f1_score} loss:{total_loss}')
    
        

In [10]:
net = PatientClassificationNet(num_feature,256)

In [11]:
train(net, trainloader, testloader, epoch=25)

epoch 0: recall: 0.8170731707317073 precision: 0.0694300518134715 f1_score: 0.12798471824259788 loss:17.13467788696289
epoch 1: recall: 0.6585365853658537 precision: 0.06553398058252427 f1_score: 0.11920529801324503 loss:10.660049438476562
epoch 2: recall: 0.6585365853658537 precision: 0.06758448060075094 f1_score: 0.12258796821793416 loss:9.973088264465332
epoch 3: recall: 0.7317073170731707 precision: 0.07453416149068323 f1_score: 0.13528748590755355 loss:9.926959991455078
epoch 4: recall: 0.7317073170731707 precision: 0.07371007371007371 f1_score: 0.13392857142857142 loss:9.246274948120117
epoch 5: recall: 0.7317073170731707 precision: 0.07731958762886598 f1_score: 0.13986013986013987 loss:8.995627403259277
epoch 6: recall: 0.8414634146341463 precision: 0.09465020576131687 f1_score: 0.17016029593094945 loss:9.025060653686523
epoch 7: recall: 0.8658536585365854 precision: 0.10502958579881656 f1_score: 0.18733509234828494 loss:8.584692001342773
epoch 8: recall: 0.8536585365853658 prec

In [12]:
correct_true = 0
predicted_true = 0
target_true = 0
total = 0
for i, data in enumerate(testloader):
    inputs, labels = data
    outputs = net(inputs)
    prediction = outputs > threshold
    correct_true += ((prediction==1) * (labels==1)).sum().item()
    target_true += labels.sum().item()
    predicted_true += prediction.sum().item()
    total += labels.size(0)
recall = correct_true / target_true
precision = correct_true / predicted_true
f1_score = 2 * precision * recall / (precision + recall)

In [13]:
print(recall, precision, f1_score)

0.6707317073170732 0.14588859416445624 0.23965141612200436


In [14]:
print(correct_true, target_true, predicted_true)

55 82.0 377


In [15]:
outputs = net(X_train)
v, _ = outputs.sort(0)
num_class = 5
class_boundary = torch.Tensor([v[int(num_train/num_class*i-1)] for i in range(1,num_class+1)])

In [16]:
out = net(X_train[0:2])
comp = out < class_boundary
# comp.nonzero()[0].item()
comp = 0

In [17]:
class PatientGroupNet(nn.Module):
    def __init__(self, patient_class_net, class_boundary):
        super(PatientGroupNet, self).__init__()
        self.patient_class_net = patient_class_net
        self.patient_class_net.eval()
        self.class_boundary = class_boundary
        self.num_class = self.class_boundary.size(0)

    def forward(self, x):
        x = self.patient_class_net(x)
        comp = x < self.class_boundary
        return self.num_class - comp.sum(-1) + 1

In [18]:
net2 = PatientGroupNet(net, class_boundary)
out = net2(X_train)
[torch.sum(out == i) for i in range(1, num_class+1)]

[tensor(850), tensor(850), tensor(847), tensor(855), tensor(852)]

In [19]:
net2 = PatientGroupNet(net, class_boundary)
out = net2(X_test)
[torch.sum(out == i) for i in range(1, num_class+1)]

[tensor(222), tensor(201), tensor(207), tensor(222), tensor(212)]

In [20]:
prediction = net2(X).numpy()
df['class'] = prediction
los_mean = [df[df['class']==i]['LOS (Days)'].mean() for i in range(1,num_class+1)]
los_mean

[0.908846193895647,
 2.4042633195598833,
 2.446816412454714,
 2.4399933614471525,
 3.5585183423172566]

In [21]:
PATH = 'D:\working\I2RL\model\test'
torch.save(net.state_dict(), PATH)

PermissionError: [Errno 13] Permission denied: 'D:\\working\\I2RL\\model'

In [488]:
X_train

tensor([[-1.4988,  1.1154, -0.3025,  ...,  1.9328, -0.1925, -0.0974],
        [-0.2893, -0.8964,  3.3051,  ..., -0.5173, -0.1925, -0.0974],
        [-1.1461,  1.1154, -0.3025,  ...,  1.9328, -0.1925, -0.0974],
        ...,
        [ 0.6681, -0.8964, -0.3025,  ..., -0.5173, -0.1925, -0.0974],
        [-1.3476,  1.1154, -0.3025,  ...,  1.9328, -0.1925, -0.0974],
        [ 0.6681,  1.1154, -0.3025,  ...,  1.9328, -0.1925, -0.0974]])